### 使用 `mpnet-base-v2` 進行檢索測試

In [ ]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# 確保使用 mpnet-base-v2 模型 (之後用BGE-M3)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2', device='cuda')

# 讀取 FAISS 索引
index = faiss.read_index("faiss_index_ivf_new.bin")

# 讀取評論數據（保留 location_id, comments）
df_comments = pd.read_csv("cleaned_comments_new.csv", usecols=["location_id", "comments"])

# 讀取地點資訊，確保 location_id 對應到地點名稱
df_info = pd.read_json("../RAW_DATA/838_location_info.json")
df_info = df_info.rename(columns={"location_id": "location_id", "gmp_location": "gmap_location"})

# 查詢文字
query_comment = ["鹿港"]
query_vector = model.encode(query_comment).astype('float32')

# 設定檢索的評論數量
k = 100  # 預設找 50 條最相似的評論
distances, indices = index.search(query_vector, k)

# 取得最相似的評論
similar_reviews = df_comments.iloc[indices[0]].copy()  # 確保 df 仍然是 DataFrame 格式

# 合併地點名稱
df_merged = similar_reviews.merge(df_info[['location_id', 'gmap_location']], on='location_id', how='left')


# 統計最相似評論對應的地點名稱出現次數
location_counts = df_merged["gmap_location"].value_counts()

# 輸出結果（Top 10 地點）
print("🔍 查詢結果（Top 10 地點）：")
print(location_counts.head(10))

# 顯示詳細評論（Top 10 條）
print("\n最相似的 10 條評論：")
for idx, row in df_merged.head(10).iterrows():
    print(f"{row['gmap_location']} - {row['comments']}")


🔍 查詢結果（Top 10 地點）：
gmap_location
鹿港天后宮             6
赤崁樓               5
安平老街              3
臺中公園              3
國定古蹟-竹塹城東門 迎曦門    2
高雄85大樓            2
臺南孔廟              2
多良觀光車站            2
觀夕平臺              2
北港朝天宮             2
Name: count, dtype: int64

最相似的 10 條評論：
三貂角燈塔 - 居高臨下鳥瞰馬崗漁港.
中正公園 - 眺望基隆港，園區廣闊
旗津老街 - 旗津渡船場 及旗津老街
松園別館 - 風景優美，瞭望花蓮港
鹿港天后宮 - 鹿港天后宮媽祖廟香火鼎盛
基隆廟口夜市 - 夜市小吃 ，近基隆港
檜意森活村 - 嘉義市休閒活動的地方
紅毛港文化園區 - 十大建設之一 高雄港
興南夜市 - 新北市知名觀光夜市。
麗寶樂園 - 麗寶秀泰影城，科莫湖


In [67]:
# 看一下所有50條評論(k = 50)長什麼樣子

print("\n最相似的 50 條評論：")
for idx, row in df_merged.head(50).iterrows():
    print(f"{row['gmap_location']} - {row['comments']}")


最相似的 50 條評論：
三貂角燈塔 - 居高臨下鳥瞰馬崗漁港.
中正公園 - 眺望基隆港，園區廣闊
旗津老街 - 旗津渡船場 及旗津老街
松園別館 - 風景優美，瞭望花蓮港
鹿港天后宮 - 鹿港天后宮媽祖廟香火鼎盛
基隆廟口夜市 - 夜市小吃 ，近基隆港
檜意森活村 - 嘉義市休閒活動的地方
紅毛港文化園區 - 十大建設之一 高雄港
興南夜市 - 新北市知名觀光夜市。
麗寶樂園 - 麗寶秀泰影城，科莫湖
半邊井 - 鹿港老街景點 半邊井
赤崁樓 - 古色古香台南市的古蹟。
永安漁港 - 永安漁港 第二類漁港
赤崁樓 - 觀光客來台南必遊之地
太武山風景區 - 登太武山，小金門之景
臺中公園 - 名勝古蹟，台中的標註，
八卦山大佛風景區 - 俯瞰彰化市區，一覽無遺
勝利星村創意生活園區 - 屏東市最有特色的一區
黃金海岸 - 台南人必來的海邊景點
鹿港天后宮 - 鹿港名勝古蹟，媽祖聖地。香火鼎盛，福澤祐民
安平老街 - 古色古香的台南必經之地
大港橋 - 駁二碼頭新景點 連接棧柒庫
高雄市文化中心 - 晨運好地方，高雄市民多加利用
基隆內港 - 基隆港～麗星郵輪～寶瓶星號
鹿港天后宮 - 鹿港人的信仰中心！香火鼎盛.
北港朝天宮 - 白沙屯2023北港進香
國定古蹟-彰化孔廟 - 彰化市孔門路（在八卦山牌樓前面）
觀夕平臺 - 台南安平觀海的好地方
蘭潭風景特定區 - 嘉義平地區的優勝美地
鵝鑾鼻燈塔 - 花蓮到台東必經的大卡點
鹿野高台 - 一年一度的台東天燈節活動
摸乳巷 - 鹿港必訪景點 讚！ 不虛此行！
多良觀光車站 - 東部歡光特色景點之一
頭城老街 - 東北角第一村落，長知識
淡水老街 - 前篇一律，台灣老街，桃園除外
駁二藝術特區 - 富文藝氣息及結合高雄渡船頭風格
大稻埕碼頭廣場 - 清晨時刻的大稻埕碼頭
花蓮觀光糖廠 - 花蓮台東休息站，種類多樣
高雄市立美術館 - 高雄市集很有水準 -森之市
湖口老街 - 竹北天后宮 — 新港奉天宮馨護台灣騎島平安活動。
內灣老街 - 新竹內灣支線的不錯景點
觀夕平臺 - 台南藏美景，海邊走走
赤崁樓 - 台南最有代表性的古蹟
三義木雕博物館 - 苗栗縣最多木雕的地方
國定古蹟-竹塹城東門 迎曦門 - 新竹地標 沒落的商圈
宜蘭昭應宮 - 宜蘭市區，三級古蹟媽祖廟
高雄85大樓 - 高雄的海景市景一次滿足
赤崁

#### 避免向量相似度低的評論過多而影響排序，加入Softmax 加權


In [6]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# mpnet-base-v2 模型)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2', device='cuda')

# 讀取 FAISS 索引
index = faiss.read_index("faiss_index_ivf_new.bin")

# 讀取評論數據（保留 location_id, comments）
df_comments = pd.read_csv("cleaned_comments_new.csv", usecols=["location_id", "comments"])

# 讀取地點資訊
df_info = pd.read_json("../RAW_DATA/838_location_info.json")
df_info = df_info.rename(columns={"location_id": "location_id", "gmp_location": "gmap_location"})

# 查詢文字
query_comment = ["黃金博物館 負面評論"]
query_vector = model.encode(query_comment).astype('float32')

# 設定檢索的評論數量
k = 100  # 預設找 50 條最相似的評論
distances, indices = index.search(query_vector, k)

# 取得最相似的評論
similar_reviews = df_comments.iloc[indices[0]].copy()  # 確保 df 仍然是 DataFrame 格式

# 合併地點名稱
df_merged = similar_reviews.merge(df_info[['location_id', 'gmap_location']], on='location_id', how='left')

# **計算原始相似度分數（避免距離為 0，+1 平滑處理）**
df_merged["similarity_score"] = 1 / (1 + distances[0])

# **應用 Softmax 加權**
alpha = 5  # 超參數（可以調整，數值越大，強化 Top 影響力）
exp_scores = np.exp(df_merged["similarity_score"] * alpha)
df_merged["weight"] = exp_scores / exp_scores.sum()

# **統計地點的加權分數**
location_scores = df_merged.groupby("gmap_location")["weight"].sum().reset_index()

# 按 `weight` 降序排序
location_scores = location_scores.sort_values(by="weight", ascending=False)

# 輸出結果（Top 10 地點）
print("🔍 查詢結果（Top 10 地點，Softmax 加權）：")
print(location_scores.reset_index(drop=True).head(10))

# 顯示詳細評論（Top 10 條）
print("\n最相似的 10 條評論（已加權）：")
for idx, row in df_merged.sort_values(by="weight", ascending=False).head(10).iterrows():
    print(f"📍 {row['gmap_location']} - {row['comments']} （權重: {row['weight']:.4f}）")

🔍 查詢結果（Top 10 地點，Softmax 加權）：
       gmap_location    weight
0          新北市立黃金博物館  0.852157
1               九份老街  0.029295
2              大甲鎮瀾宮  0.019841
3              東港東隆宮  0.010912
4  東港漁港漁產品直銷中心（華僑市場）  0.010861
5            國立故宮博物院  0.010703
6               黃金海岸  0.010171
7            國立中正紀念堂  0.009915
8              二延平步道  0.009415
9              艋舺龍山寺  0.009361

最相似的 10 條評論（已加權）：
📍 新北市立黃金博物館 - 금광이 폐광되고 만든 황금박물관. 그냥 있으니 한번 지나면서 볼만합니다. （權重: 0.0132）
📍 新北市立黃金博物館 - ⛩黃金博物館 - 📷好攝程度：💛💛🖤🖤🖤 - 看看摸了個黃金 荷包會不會多點黃金👝 （權重: 0.0130）
📍 新北市立黃金博物館 - 博物館外没有標示黃金博物館的字樣?門票80元新北市民入場不用錢，裡面值得看只有大塊黃金而已 （權重: 0.0122）
📍 新北市立黃金博物館 - 金瓜石看海摸黃金 #黃金博物館看展覽 #摸4億黃金求好運 #招財黃金貓看了歡喜 #太子賓館建於1920年 #四連棟日式宿舍 #斜坡索道，#十三層遺址 #基隆山及大肚美人山 #台灣文化日免門票 #黃金博物館門票每人80 隨時都想要去黃金博物館要去摸一下黃金，而且今年是牛年更希望扭轉乾坤，更加順利達成目標，鄰居剛好有空，就一起去祈運了！摸黃金的感覺真的很棒，難怪這麼想來，再者今天是台灣文化日，所以參觀黃金博物館共四個館，摸黃金，全部免門票，也未免太幸運了！所以今天除了在全家買了一杯拿鐵只要$35 ，今天總計花35 ，台灣的旅行也未免太便宜了！ 2020年10月17日下午兩點多就想到海岸邊看海，望海，金瓜石摸黃金，心情變更好😍 金瓜石這一整個區域大概有20幾個地點可以經常來這裡走走，每次就選個三四個地方，深入去逛逛看看研究了解這裡的文化，感覺收穫滿滿！ （權重: 0.

In [7]:
# 看一下所有50條評論(k = 50)長什麼樣子

print("\n最相似的 50 條評論（已加權）：") 
for idx, row in df_merged.sort_values(by="weight", ascending=False).head(100).iterrows():
    print(f"📍 {row['gmap_location']} - {row['comments']} （權重: {row['weight']:.4f}）")


最相似的 50 條評論（已加權）：
📍 新北市立黃金博物館 - 금광이 폐광되고 만든 황금박물관. 그냥 있으니 한번 지나면서 볼만합니다. （權重: 0.0132）
📍 新北市立黃金博物館 - ⛩黃金博物館 - 📷好攝程度：💛💛🖤🖤🖤 - 看看摸了個黃金 荷包會不會多點黃金👝 （權重: 0.0130）
📍 新北市立黃金博物館 - 博物館外没有標示黃金博物館的字樣?門票80元新北市民入場不用錢，裡面值得看只有大塊黃金而已 （權重: 0.0122）
📍 新北市立黃金博物館 - 金瓜石看海摸黃金 #黃金博物館看展覽 #摸4億黃金求好運 #招財黃金貓看了歡喜 #太子賓館建於1920年 #四連棟日式宿舍 #斜坡索道，#十三層遺址 #基隆山及大肚美人山 #台灣文化日免門票 #黃金博物館門票每人80 隨時都想要去黃金博物館要去摸一下黃金，而且今年是牛年更希望扭轉乾坤，更加順利達成目標，鄰居剛好有空，就一起去祈運了！摸黃金的感覺真的很棒，難怪這麼想來，再者今天是台灣文化日，所以參觀黃金博物館共四個館，摸黃金，全部免門票，也未免太幸運了！所以今天除了在全家買了一杯拿鐵只要$35 ，今天總計花35 ，台灣的旅行也未免太便宜了！ 2020年10月17日下午兩點多就想到海岸邊看海，望海，金瓜石摸黃金，心情變更好😍 金瓜石這一整個區域大概有20幾個地點可以經常來這裡走走，每次就選個三四個地方，深入去逛逛看看研究了解這裡的文化，感覺收穫滿滿！ （權重: 0.0120）
📍 新北市立黃金博物館 - 황금을 드는자 근손실이 오리리 （權重: 0.0119）
📍 新北市立黃金博物館 - 挖掘黄金之旅的历史，有一块2百多公斤的黄金在展馆。很无聊 （權重: 0.0117）
📍 新北市立黃金博物館 - 일제 시대 금광 수탈의 고통이 남아 있는 곳 （權重: 0.0116）
📍 新北市立黃金博物館 - 要發財多來黃金博物館摸黃金 （權重: 0.0115）
📍 新北市立黃金博物館 - 了解金矿历史文化，可以摸黄金。门票80台币。2018.06.19 （權重: 0.0115）
📍 新北市立黃金博物館 - 금 말고는 볼 것리 없음 ㅠㅠ （權重: 0.0114）
📍 新北市立黃金博物館 - 尋訪黃金城歷史…摸摸上億元的黃金塊 （權重: 0.0113）
📍 新北市立黃金博物館 - 황금박물관. 황금

### 可以指定語言、地點來做語意向量檢索

In [2]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# 載入 mpnet-base-v2 模型 (或 bge-m3)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2', device='cuda')

# **讀取 FAISS 原索引**
index = faiss.read_index("faiss_index_ivf_new.bin")

# **讀取評論數據**
df_comments = pd.read_csv("cleaned_comments_new.csv", usecols=["review_id", "location_id", "comments", "language"])

# **讀取地點資訊**
df_info = pd.read_json("../RAW_DATA/838_location_info.json")
df_info = df_info.rename(columns={"location_id": "location_id", "gmap_location": "gmap_location"})

# **讀取 review_id.npy**
review_ids = np.load("review_ids_new.npy")  # 原始 FAISS 索引對應的 review_id

# **使用者可指定 location_id & 語言**
selected_locations = []  # 這裡填入要篩選的地點
selected_languages = []  # 可篩選多種語言

# **篩選符合條件的評論**
if selected_locations:
    df_comments = df_comments[df_comments["location_id"].isin(selected_locations)]
if selected_languages:
    df_comments = df_comments[df_comments["language"].isin(selected_languages)]


# **如果 `df_comments` 為空，避免 FAISS 檢索**
if df_comments.empty:
    print("🚨 沒有符合條件的評論，請調整篩選條件！")
    raise Exception("篩選條件過於嚴格，沒有找到符合的評論。")

# **查詢文字**
query_comment = ["豐富館藏"]
query_vector = model.encode(query_comment).astype('float32')

# **設定檢索評論數量**
k = min(100, len(df_comments))

# **FAISS 檢索**
distances, indices = index.search(query_vector, k)

# **使用 review_id 來確保索引對應**
retrieved_review_ids = [review_ids[i] for i in indices[0]]

# **篩選匹配的評論**
similar_reviews = df_comments[df_comments["review_id"].isin(retrieved_review_ids)].copy()

# **合併地點名稱**
df_merged = similar_reviews.merge(df_info[['location_id', 'gmap_location']], on='location_id', how='left')

# **計算原始相似度分數（避免距離為 0，+1 平滑處理）**
df_merged["similarity_score"] = 1 / (1 + distances[0][:len(similar_reviews)])

# **應用 Softmax 加權**
alpha = 5
exp_scores = np.exp(df_merged["similarity_score"] * alpha)
df_merged["weight"] = exp_scores / exp_scores.sum()

# **統計地點的加權分數**
location_scores = df_merged.groupby("gmap_location")["weight"].sum().reset_index()

# **按 `weight` 降序排序**
location_scores = location_scores.sort_values(by="weight", ascending=False)

# **輸出結果（Top 10 地點）**
print("\n🔍 查詢結果（經過 Softmax 加權）：")
print("=" * 60)
for idx, row in location_scores.reset_index(drop=True).head(10).iterrows():
    print(f"{idx+1:2d}. 📍 {row['gmap_location']:30s} | 🔥 加權分數: {row['weight']:.4f}")

print("\n📝 最相似的 10 條評論：")
print("=" * 60)
for idx, row in df_merged.sort_values(by="weight", ascending=False).head(10).iterrows():
    print(f"📍 {row['gmap_location']}")
    print(f"💬 {row['comments']}")
    print(f"⚖️  相似度權重: {row['weight']:.4f}")
    print("-" * 60)



🔍 查詢結果（經過 Softmax 加權）：
 1. 📍 奇美博物館                          | 🔥 加權分數: 0.4700
 2. 📍 國立故宮博物院                        | 🔥 加權分數: 0.2914
 3. 📍 三義木雕博物館                        | 🔥 加權分數: 0.0372
 4. 📍 新竹市影像博物館                       | 🔥 加權分數: 0.0314
 5. 📍 淡水紅毛城                          | 🔥 加權分數: 0.0283
 6. 📍 國立自然科學博物館                      | 🔥 加權分數: 0.0194
 7. 📍 台北101購物中心                      | 🔥 加權分數: 0.0154
 8. 📍 國立故宮博物院南部院區                    | 🔥 加權分數: 0.0151
 9. 📍 國立臺灣美術館                        | 🔥 加權分數: 0.0086
10. 📍 新北市立十三行博物館                     | 🔥 加權分數: 0.0077

📝 最相似的 10 條評論：
📍 新竹市影像博物館
💬 稀有的古董物件長知識了
⚖️  相似度權重: 0.0314
------------------------------------------------------------
📍 淡水紅毛城
💬 古績工法精緻收藏很多古物
⚖️  相似度權重: 0.0283
------------------------------------------------------------
📍 國立自然科學博物館
💬 館藏豐富 師生必備良去
⚖️  相似度權重: 0.0194
------------------------------------------------------------
📍 三義木雕博物館
💬 物藏豐富，值得停留欣賞
⚖️  相似度權重: 0.0190
------------------------------------------------------------
📍 三義木雕

In [14]:
print("\n📝 最相似的 20 條評論（已加權）：")
print("=" * 60)
for idx, row in df_merged.sort_values(by="weight", ascending=False).head(20).iterrows():
    print(f"📍 {row['gmap_location']}")
    print(f"💬 {row['comments']}")
    print(f"⚖️  相似度權重: {row['weight']:.4f}")
    print("-" * 60)


📝 最相似的 20 條評論（已加權）：
📍 新北市立黃金博物館
💬 생각보다는 별로였음~~
⚖️  相似度權重: 0.5494
------------------------------------------------------------
📍 新北市立黃金博物館
💬 볼건 크게 없었어요^^
⚖️  相似度權重: 0.4506
------------------------------------------------------------


#### **如果以越南文(透過google翻譯)來執行query，會和中文執行query結果不同，需要想辦法解決**

### 使用 `BGE-M3` 進行檢索測試

In [15]:
import faiss
import numpy as np
import pandas as pd
from FlagEmbedding import BGEM3FlagModel

# 🔹 使用 BGE-M3 模型（確保使用 dense_vecs）
model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=True, use_cuda=True, device=0)

# 🔹 讀取 FAISS 索引
index = faiss.read_index("faiss_index_ivf_bge.bin")

# 🔹 讀取評論數據（保留 location_ID, comments）
df_comments = pd.read_csv("cleaned_comments.csv", usecols=["location_ID", "comments"])

# 🔹 讀取地點資訊，確保 location_ID 對應到地點名稱
df_info = pd.read_json("../RAW_DATA/838_location_info.json")
df_info = df_info.rename(columns={"location_id": "location_ID", "gmp_location": "gmap_location"})

# 🔹 查詢文字
query_comment = ["可以看到夕陽 海邊的咖啡廳"]
query_results = model.encode(query_comment)

# BGE-M3 會回傳 dict，確保取出 `dense_vecs` (之後要加入另外兩個'lexical_weights', 'colbert_vectors')
if isinstance(query_results, dict) and "dense_vecs" in query_results:
    query_vector = np.array(query_results["dense_vecs"]).astype('float32')
else:
    raise ValueError(" `model.encode()` 沒有return `dense_vecs` ")

# 🔹 設定檢索的評論數量
k = 50  # 預設找 50 條最相似的評論
distances, indices = index.search(query_vector, k)

# 🔹 取得最相似的評論
similar_reviews = df_comments.iloc[indices[0]].copy()  # 確保 df 仍然是 DataFrame 格式

# 🔹 合併地點名稱
df_merged = similar_reviews.merge(df_info[['location_ID', 'gmap_location']], on='location_ID', how='left')

# 🔹 統計最相似評論對應的地點名稱出現次數
location_counts = df_merged["gmap_location"].value_counts()

# 🔹 輸出結果（Top 10 地點）
print("🔍 查詢結果（Top 10 地點）：")
print(location_counts.head(10))

# 🔹 顯示詳細評論（Top 10 條）
print("\n最相似的 10 條評論：")
for idx, row in df_merged.head(10).iterrows():
    print(f"{row['gmap_location']} - {row['comments']}")


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


🔍 查詢結果（Top 10 地點）：
gmap_location
九份老街                                                    15
臺南孔廟                                                     7
六福村主題遊樂園                                                 6
奇美博物館                                                    3
國立故宮博物院                                                  3
卑南遺址公園 Peinan Site Park (Peinan Archaeological Park)     2
東眼山國家森林遊樂區                                               2
七星潭海岸風景特定區                                               2
瑞豐夜市                                                     2
台北101購物中心                                                1
Name: count, dtype: int64

最相似的 10 條評論：
劍湖山世界主題樂園 - 小朋友玩的很開心，但咕咕列車排隊很久小朋友不耐煩。
六福村主題遊樂園 - 園區販售的東西真不是普通的貴
興南夜市 - 乾淨，很好逛
東眼山國家森林遊樂區 - 1.山區氣候涼爽， 2.主要登山步道有二，每條步道約需2.5小時左右，步道內空氣宜人，消暑，且幾乎曬不到陽光
奇美博物館 - 館內常態展很值得一看再看，南市民應該多多品嚐，戶外適合讓小孩放電，大人們找個舒服的地方席地而坐聊天野餐也不用自帶喇叭
國立故宮博物院 - Incredible museum with a collection that is breathtaking and simply world-class. The artefacts exposed cover thous

In [16]:
print("\n最相似的 50 條評論：")
for idx, row in df_merged.head(50).iterrows():
    print(f"{row['gmap_location']} - {row['comments']}")


最相似的 50 條評論：
劍湖山世界主題樂園 - 小朋友玩的很開心，但咕咕列車排隊很久小朋友不耐煩。
六福村主題遊樂園 - 園區販售的東西真不是普通的貴
興南夜市 - 乾淨，很好逛
東眼山國家森林遊樂區 - 1.山區氣候涼爽， 2.主要登山步道有二，每條步道約需2.5小時左右，步道內空氣宜人，消暑，且幾乎曬不到陽光
奇美博物館 - 館內常態展很值得一看再看，南市民應該多多品嚐，戶外適合讓小孩放電，大人們找個舒服的地方席地而坐聊天野餐也不用自帶喇叭
國立故宮博物院 - Incredible museum with a collection that is breathtaking and simply world-class. The artefacts exposed cover thousands of years of history in many materials such as bronze ceramics or jade. The museum is understandably mobbed by large tour groups - be prepared to skip sections and come back to them later. The good news is there are so many treasures in here that you can always find your way to an artefact you can inspect all by yourself for a little while. The museum does a fabulous work at education beyond just exposing nice objects - it really taught me to look at jade pieces differently. Nice museum shop.
九份老街 - 야경 그저 그랬음. 버스투어 혹은 택시투어 없이 대중교통을 이용해서 간다면 꽤나 험난함!(대중교통 이용객이 워낙 많아서... 도로가 왕복 2차선인 것도 한몫합니다) 엄청 기대하고 환상 가지면 실망도 클 겁니다.
六福村主題遊樂園 - 很喜歡這裡的表演，很歡

#### 簡直垃圾，計畫把另外兩個模型吐出來的東西加入檢索機制'lexical_weights', 'colbert_vectors'